In [1]:
import pandas as pd
import tensorflow as tf
from time import time
print(tf.__version__)

1.11.0


In [2]:
df_train = pd.read_csv("/data/MNIST/mnist_train.csv", header=None)
df_test = pd.read_csv("/data/MNIST/mnist_test.csv", header=None)

In [3]:
X_train = df_train.iloc[:, 1:].values/255
X_test = df_test.iloc[:, 1:].values/255

y_train = df_train.iloc[:, 0].values
y_test = df_test.iloc[:, 0].values

In [4]:
def batching(X_data, y_data, session, epochs = 10, batch_size = 128):

    X_placeholder = tf.placeholder(X_data.dtype, X_data.shape)
    y_placeholder = tf.placeholder(y_data.dtype, y_data.shape)

    dataset = tf.data.Dataset.from_tensor_slices((X_placeholder, y_placeholder))
    dataset = dataset.shuffle(X_data.shape[0])
    dataset = dataset.repeat(epochs)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(batch_size * 100)

    iterator = dataset.make_initializable_iterator()
    next_batch = iterator.get_next()

    sess.run(iterator.initializer
             , feed_dict={X_placeholder: X_data, y_placeholder: y_data})

    while True:
        try:
            yield sess.run(next_batch)
        except tf.errors.OutOfRangeError:
            break

In [5]:
tf.reset_default_graph()


X = tf.placeholder("float32", (None, 784))
y = tf.placeholder("int32", (None))

y_oh = tf.one_hot(y, depth=10)

Z = X
Z = tf.layers.Dense(400, activation="relu", kernel_initializer="he_normal")(Z)
Z = tf.layers.Dropout(rate = 0.5)(Z)
Z = tf.layers.Dense(100, activation="relu", kernel_initializer="he_normal")(Z)
Z = tf.layers.Dropout(rate = 0.5)(Z)
Z = tf.layers.Dense(10)(Z)

y_prob = tf.nn.softmax(Z)
y_pred = tf.argmax(Z, axis=1, output_type="int32")

match = tf.equal(y, y_pred)
accuracy = tf.reduce_mean(tf.cast(match, "float32"))

loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_oh, logits=Z)
cost = tf.reduce_mean(loss)

opt = tf.train.AdamOptimizer(1e-3).minimize(cost)

saver = tf.train.Saver()
init = tf.global_variables_initializer()
tf.summary.scalar("cost", cost)
tf.summary.scalar("accuracy", accuracy)

summaries = tf.summary.merge_all()
file_writer = tf.summary.FileWriter("/tmp/tf/" + str(time()))

config = tf.ConfigProto(intra_op_parallelism_threads=12
                        , inter_op_parallelism_threads=12)

with tf.Session(config=config) as sess:
    sess.run(init)
    
    i = 0
    for X_batch, y_batch in batching(X_train, y_train, sess, epochs=10, batch_size=32):
        cost_, accuracy_, op_, summaries_ = sess.run([cost, accuracy, opt, summaries]
                    , feed_dict={X:X_batch, y: y_batch})
        if i % 100 == 0:
            file_writer.add_summary(summaries_)
            print("%d %.4f" % (i, accuracy_), end="\r")
        i += 1
    print("Test accuracy:", sess.run(accuracy, feed_dict={X:X_test, y:y_test}))

Test accuracy: 0.9826


In [6]:
32 * 1800

57600

In [7]:
X_train_img = X_train.reshape(-1, 28,28,1)
X_test_img = X_test.reshape(-1, 28,28,1)

In [9]:
tf.reset_default_graph()


X = tf.placeholder("float32", (None, 28, 28, 1))
y = tf.placeholder("int32", (None))

y_oh = tf.one_hot(y, depth=10)

Z = X
Z = tf.layers.Conv2D(32, (3, 3), activation="relu", kernel_initializer="he_normal")(Z)
Z = tf.layers.MaxPooling2D((2,2), (1,1))(Z)
Z = tf.layers.Dropout(rate = 0.5)(Z)
Z = tf.layers.Conv2D(64, (5, 5), activation="relu", kernel_initializer="he_normal")(Z)
Z = tf.layers.MaxPooling2D((2,2), (1,1))(Z)
Z = tf.layers.Dropout(rate = 0.5, )(Z)
Z = tf.layers.Flatten()(Z)
Z = tf.layers.Dense(400, activation="relu", kernel_initializer="he_normal")(Z)
Z = tf.layers.Dense(100, activation="relu", kernel_initializer="he_normal")(Z)
Z = tf.layers.Dense(10)(Z)

y_prob = tf.nn.softmax(Z)
y_pred = tf.argmax(Z, axis=1, output_type="int32")

match = tf.equal(y, y_pred)
accuracy = tf.reduce_mean(tf.cast(match, "float32"))

loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_oh, logits=Z)
cost = tf.reduce_mean(loss)

opt = tf.train.AdamOptimizer(1e-3).minimize(cost)

saver = tf.train.Saver()
init = tf.global_variables_initializer()
tf.summary.scalar("cost", cost)
tf.summary.scalar("accuracy", accuracy)

summaries = tf.summary.merge_all()
file_writer = tf.summary.FileWriter("/tmp/tf/" + str(int(time())))

config = tf.ConfigProto(intra_op_parallelism_threads=12
                        , inter_op_parallelism_threads=12)

with tf.Session(config=config) as sess:
    sess.run(init)
    
    i = 0
    for X_batch, y_batch in batching(X_train_img, y_train, sess, epochs=3, batch_size=32):
        cost_, accuracy_, op_, summaries_ = sess.run([cost, accuracy, opt, summaries]
                    , feed_dict={X:X_batch, y: y_batch})
        if i % 100 == 0:
            #file_writer.add_summary(summaries_)
            print("%d %.4f" % (i, accuracy_), end="\r")
        i += 1
    print("Test accuracy:", sess.run(accuracy, feed_dict={X:X_test_img, y:y_test}))

Test accuracy: 0.9862
